In [0]:
weather = spark.table("workspace.default.weather_europe")
generation = spark.table("workspace.schema_capstone.generation_clean")
load = spark.table("curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw__v2.load_actual")
crossborder = spark.table("curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw__v2.crossborder_flows")


In [0]:
print("load:", load.columns)
print("generation:", generation.columns)
print("weather:", weather.columns)
print("crossborder flows:", weather.columns)

In [0]:
# need to rename timestamp column to index
weather = weather.withColumnRenamed("timestamp", "index")
crossborder = crossborder.withColumnRenamed("timestamp", "index")

In [0]:
num_rows, num_columns = load.count(), len(load.columns)
print(f"Rows: {num_rows}, Columns: {num_columns}")

In [0]:
num_rows, num_columns = generation.count(), len(generation.columns)
print(f"Rows: {num_rows}, Columns: {num_columns}")

In [0]:
num_rows, num_columns = crossborder.count(), len(crossborder.columns)
print(f"Rows: {num_rows}, Columns: {num_columns}")

In [0]:
from pyspark.sql import functions as F

# for load dataframe

# Identify numeric columns in load
numeric_cols = [
    c for c, t in load.dtypes
    if t in ("double", "float", "int", "bigint")
]

# Truncate timestamp to hour
load = load.withColumn("hour", F.date_trunc("hour", F.col("index")))

# Aggregate by country and hour
agg_exprs = [F.mean(F.col(c)).alias(c) for c in numeric_cols]

load_hourly = load.groupBy("country", "hour").agg(*agg_exprs).orderBy("hour")

load_hourly = load_hourly.withColumnRenamed("hour", "index")
display(load_hourly)


In [0]:
num_rows, num_columns = load_hourly.count(), len(load_hourly.columns)
print(f"Rows: {num_rows}, Columns: {num_columns}")

In [0]:
display(crossborder)

In [0]:
# for crossborder dataframe

# Identify numeric columns in crossborder
dtypes = crossborder.dtypes
numeric_cols = [c for c, t in dtypes if t in ("double", "float", "int", "bigint")]

# Truncate timestamp to hour
crossborder = crossborder.withColumn("hour", F.date_trunc("hour", F.col("index")))

# Aggregate by from_country, to_country, and hour
agg_exprs = [F.mean(F.col(c)).alias(c) for c in numeric_cols]
crossborder_hourly = crossborder.groupBy("from_country", "to_country", "hour").agg(*agg_exprs).orderBy("hour")

# Rename hour column to index
crossborder_hourly = crossborder_hourly.withColumnRenamed("hour", "index")
display(crossborder_hourly)

In [0]:
num_rows, num_columns = crossborder_hourly.count(), len(crossborder_hourly.columns)
print(f"Rows: {num_rows}, Columns: {num_columns}")

In [0]:
# Net flow per country
imports = crossborder_hourly.groupBy("to_country", "index").agg(F.sum("Value").alias("import_mw"))
exports = crossborder_hourly.groupBy("from_country", "index").agg(F.sum("Value").alias("export_mw"))

In [0]:
# Rename index columns before join
imports = imports.withColumnRenamed("index", "import_index")
exports = exports.withColumnRenamed("index", "export_index")

# Join imports and exports for each country and timestamp
net_flow = imports.join(
    exports,
    (imports["to_country"] == exports["from_country"]) & (imports["import_index"] == exports["export_index"]),
    how="full_outer"
).fillna(0)

In [0]:
# Compute net imports
net_flow = net_flow.withColumn(
    "index",
    F.coalesce("import_index", "export_index")
).withColumn(
    "country",
    F.coalesce("to_country", "from_country")
).withColumn(
    "net_imports",
    F.col("import_mw") - F.col("export_mw")
).select(
    "country", "index", "net_imports"
)

In [0]:
display(net_flow)

In [0]:
num_rows, num_columns = net_flow.count(), len(net_flow.columns)
print(f"Rows: {num_rows}, Columns: {num_columns}")

In [0]:
# since we have less timestamps for load_hourly, we will join to this table
df = load_hourly
df = df.join(generation, on=["index", "country"], how="inner")
df = df.join(net_flow, on=["index", "country"], how="inner")
df = df.join(weather, on=["index", "country"], how="inner")
df = df.drop("month")


In [0]:
num_rows, num_columns = df.count(), len(df.columns)
print(f"Rows: {num_rows}, Columns: {num_columns}")

In [0]:
display(df.orderBy("index"))

In [0]:
df.write.mode("overwrite").saveAsTable("electricity_and_weather_europe")

In [0]:
df_nulls = df.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns
])

# Convert to table: column_name | null_count
expr = ", ".join([f"'{c}', {c}" for c in df.columns])
df_nulls_long = df_nulls.selectExpr(f"stack({len(df.columns)}, {expr}) as (column_name, null_count)")

df_nulls_long.show(truncate=False)